# Using autodiff to check gradient/Hessians

In [1]:
using Revise
using DataFrames, Random, GLM, QuasiCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using ToeplitzMatrices
using BenchmarkTools
using SnpArrays
using ForwardDiff
# using MendelPlots

BLAS.set_num_threads(1)
Threads.nthreads()

function simulate_random_snparray(s::Union{String, UndefInitializer}, n::Int64,
    p::Int64; mafs::Vector{Float64}=zeros(Float64, p), min_ma::Int = 5)

    #first simulate a random {0, 1, 2} matrix with each SNP drawn from Binomial(2, r[i])
    A1 = BitArray(undef, n, p) 
    A2 = BitArray(undef, n, p) 
    for j in 1:p
        minor_alleles = 0
        maf = 0
        while minor_alleles <= min_ma
            maf = 0.5rand()
            for i in 1:n
                A1[i, j] = rand(Bernoulli(maf))
                A2[i, j] = rand(Bernoulli(maf))
            end
            minor_alleles = sum(view(A1, :, j)) + sum(view(A2, :, j))
        end
        mafs[j] = maf
    end

    #fill the SnpArray with the corresponding x_tmp entry
    return _make_snparray(s, A1, A2)
end

function _make_snparray(s::Union{String, UndefInitializer}, A1::BitArray, A2::BitArray)
    n, p = size(A1)
    x = SnpArray(s, n, p)
    for i in 1:(n*p)
        c = A1[i] + A2[i]
        if c == 0
            x[i] = 0x00
        elseif c == 1
            x[i] = 0x02
        elseif c == 2
            x[i] = 0x03
        else
            throw(MissingException("matrix shouldn't have missing values!"))
        end
    end
    return x
end

┌ Info: Precompiling QuasiCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc]
└ @ Base loading.jl:1423


_make_snparray (generic function with 1 method)

In [2]:
function simulate_VC_longitudinal(;
    n = 1000, # sample size
    d = 5, # number of observations per sample
    p = 3, # number of nongenetic covariates, including intercept
    m = 2, # number of variance components
    q = 1000, # number of SNPs
    k = 10, # number of causal SNPs
    seed = 2022,
    y_distribution = Bernoulli,
    T = Float64,
    )
    m == 1 || m == 2 || error("m (number of VC) must be 1 or 2")
    
    # non-genetic effect sizes
    Random.seed!(seed)
    βtrue = rand(Uniform(-0.2, 0.2), p)
    dist = y_distribution()
    link = canonicallink(dist)
    Dist = typeof(dist)
    Link = typeof(link)

    # variance components
    θtrue = fill(0.1, m)
    V1 = ones(d, d)
    V2 = Matrix(I, d, d)
    Γ = m == 1 ? θtrue[1] * V1 : θtrue[1] * V1 + θtrue[2] * V2

    # simulate design matrices
    Random.seed!(seed)
    X_full = [hcat(ones(d), randn(d, p - 1)) for i in 1:n]

    # simulate random SnpArray with 100 SNPs and randomly choose k SNPs to be causal
    Random.seed!(2022)
    G = simulate_random_snparray(undef, n, q)
    Gfloat = convert(Matrix{T}, G, center=true, scale=false)
    γtrue = zeros(q)
    γtrue[1:k] .= rand([-0.2, 0.2], k)
    shuffle!(γtrue)
    η_G = Gfloat * γtrue

    # simulate phenotypes
    if y_distribution == Normal
        τtrue = 10.0
        σ2 = inv(τtrue)
        σ = sqrt(σ2)
        obs = Vector{GaussianCopulaVCObs{T}}(undef, n)
        for i in 1:n
            X = X_full[i]
            η = X * βtrue
            η .+= η_G[i] # add genetic effects
            μ = GLM.linkinv.(link, η)
            vecd = Vector{ContinuousUnivariateDistribution}(undef, d)
            for i in 1:d
                vecd[i] = y_distribution(μ[i], σ)
            end
            nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
            # simuate single vector y
            y = Vector{T}(undef, d)
            res = Vector{T}(undef, d)
            rand(nonmixed_multivariate_dist, y, res)
            V = m == 1 ? [V1] : [V1, V2]
            obs[i] = GaussianCopulaVCObs(y, X, V)
        end
        qc_model = GaussianCopulaVCModel(obs)
    else
        obs = Vector{GLMCopulaVCObs{T, Dist, Link}}(undef, n)
        for i in 1:n
            X = X_full[i]
            η = X * βtrue
            η .+= η_G[i] # add genetic effects
            μ = GLM.linkinv.(link, η)
            vecd = Vector{DiscreteUnivariateDistribution}(undef, d)
            for i in 1:d
                vecd[i] = y_distribution(μ[i])
            end
            nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
            # simuate single vector y
            y = Vector{T}(undef, d)
            res = Vector{T}(undef, d)
            rand(nonmixed_multivariate_dist, y, res)
            V = m == 1 ? [V1] : [V1, V2]
            obs[i] = GLMCopulaVCObs(y, X, V, dist, link)
        end
        qc_model = GLMCopulaVCModel(obs)
    end
    return qc_model, Γ, G, βtrue, θtrue, γtrue
end

k = 0 # number of causal SNPs

qc_model, Γ, G, βtrue, θtrue, γtrue = simulate_VC_longitudinal(
    n = 5000, # sample size
    d = 5, # number of observations per sample
    p = 3, # number of fixed effects, including intercept
    m = 1, # number of variance components
    q = 0, # number of SNPs
    k = k, # number of causal SNPs
    seed = 1000,
    y_distribution = Bernoulli,
    T = Float64,
)

@show qc_model;

qc_model = Quasi-Copula Variance Component Model
  * base distribution: Bernoulli
  * link function: LogitLink
  * number of clusters: 5000
  * cluster size min, max: 5, 5
  * number of variance components: 1
  * number of fixed effects: 3



In [5]:
@time optm = QuasiCopula.fit!(qc_model,
    Ipopt.IpoptSolver(
        print_level = 5, 
        tol = 10^-6, 
        max_iter = 1000,
        accept_after_max_steps = 4,
        warm_start_init_point="yes", 
        limited_memory_max_history = 6, # default value
        hessian_approximation = "limited-memory",
#         derivative_test="second-order"
    )
);

initializing β using Newton's Algorithm under Independence Assumption
gcm.β = [0.026499669081559256, -0.011594461794217123, 0.06008621943931356]
initializing variance components using MM-Algorithm
gcm.θ = [0.10572650177558549]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds

In [6]:
@show βtrue
@show qc_model.β
@show qc_model.∇β

@show θtrue
@show qc_model.θ
@show qc_model.∇θ;

βtrue = [0.019926508247760877, 0.010309257172965214, 0.05956897900629837]
qc_model.β = [0.02649195678310015, -0.010667144141669664, 0.06480115759419805]
qc_model.∇β = [1.0586084292496523e-7, -5.6072888887648276e-8, -4.622918350072336e-7]
θtrue = [0.1]
qc_model.θ = [0.10577520258371464]
qc_model.∇θ = [-5.719336200193936e-7]


## Is $\nabla_\beta res$ calculated correctly? 

We can check using ForwardDiff

The function is 

$$res_{ij}(\beta) = \frac{y_i - \mu_i}{\sigma_{ij}(\beta)}$$

### Normal

Assumes y, X are given. We calculate the residuals for just 1 sample

In [2]:
# sample data
X = qc_model.data[1].X # d by p
y = qc_model.data[1].y # d by 1

# objective
function resβ(β)
    η = X * β # d by 1
    μ = GLM.linkinv.(IdentityLink(), η)
    varμ = GLM.glmvar.(Normal(), μ)
    return (y - μ) ./ sqrt.(varμ) # d by 1
end

# mathematical gradient
function ∇resβ(β)
    d, p = size(X)
    ∇resβ = zeros(d, p)
    for i in 1:p, j in 1:d
        ∇resβ[j, i] = -X[j, i]
    end
    return ∇resβ # d × p
end

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)

# random beta vector
β = rand(size(qc_model.data[1].X, 2))

# check objective
@show resβ(β)

# compare mathematical and numerical gradient
[vec(∇resβ(β)) vec(∇resβ_autodiff(β))]

resβ(β) = [-1.5263015405222384, -2.6945001310537258, -1.9847678519577736, -0.900074590336336]


12×2 Matrix{Float64}:
 -1.0        -1.0
 -1.0        -1.0
 -1.0        -1.0
 -1.0        -1.0
  2.07458     2.07458
 -1.94686    -1.94686
  0.0808759   0.0808759
  0.154606    0.154606
 -0.931964   -0.931964
 -2.26098    -2.26098
 -1.19819    -1.19819
  0.0763038   0.0763038

### Bernoulli

In [3]:
# sample data
X = qc_model.data[1].X # d by p
y = qc_model.data[1].y # d by 1

# objective
function resβ(β)
    η = X * β # d by 1
    μ = GLM.linkinv.(LogitLink(), η)
    varμ = GLM.glmvar.(Bernoulli(), μ)
    return (y - μ) ./ sqrt.(varμ) # d by 1
end

# mathematical gradient
function ∇resβ(β)
    d, p = size(X)
    ∇resβ = zeros(d, p)
    η = X * β # d by 1
    μ = GLM.linkinv.(LogitLink(), η) # d by 1
    varμ = GLM.glmvar.(Bernoulli(), μ) # d by 1
    res = (y - μ) ./ sqrt.(varμ) # d by 1
    for i in 1:p, j in 1:d
        varμ_j = varμ[j]
        x_ji = X[j, i]
        res_j = res[j]
        μ_j = μ[j]
        ∇resβ[j, i] = -sqrt(varμ_j) * x_ji - (0.5 * res_j * (1 - 2μ_j) * x_ji)
    end
    return ∇resβ # d × p
end

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)

# random beta vector
β = rand(size(qc_model.data[1].X, 2))

# check objective
@show resβ(β)

# compare mathematical and numerical gradient
[vec(∇resβ(β)) vec(∇resβ_autodiff(β))]

resβ(β) = [1.3943163199970943, 0.24515130649692646, 0.6580775661179072, 0.6577805352369949, 0.390446316792847]


15×2 Matrix{Float64}:
 -0.697158   -0.697158
 -0.122576   -0.122576
 -0.329039   -0.329039
 -0.32889    -0.32889
 -0.195223   -0.195223
  1.44631     1.44631
 -0.238638   -0.238638
  0.0266113   0.0266113
  0.0508486   0.0508486
 -0.181941   -0.181941
 -1.57626    -1.57626
 -0.146869   -0.146869
  0.0251069   0.0251069
 -0.154771   -0.154771
 -0.201156   -0.201156

### Poisson

In [61]:
# sample data
X = qc_model.data[1].X # d by p
y = qc_model.data[1].y # d by 1

# objective
function resβ(β)
    η = X * β # d by 1
    μ = GLM.linkinv.(LogLink(), η)
    varμ = GLM.glmvar.(Poisson(), μ)
    return (y - μ) ./ sqrt.(varμ) # d by 1
end

# mathematical gradient
function ∇resβ(β)
    d, p = size(X)
    ∇resβ = zeros(d, p)
    η = X * β # d by 1
    μ = GLM.linkinv.(LogLink(), η) # d by 1
    varμ = GLM.glmvar.(Poisson(), μ) # d by 1
    res = (y - μ) ./ sqrt.(varμ) # d by 1
    dμ = GLM.mueta.(LogLink(), η) # d by 1
    for i in 1:p, j in 1:d
        varμ_j = varμ[j]
        x_ji = X[j, i]
        res_j = res[j]
        μ_j = μ[j]
        dμ_j = dμ[j]
        ∇resβ[j, i] = x_ji * (-(inv(sqrt(varμ_j)) + (0.5 * inv(varμ_j)) * res_j) * dμ_j)
    end
    return ∇resβ # d × p
end

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)

# random beta vector
β = rand(size(qc_model.data[1].X, 2))

# check objective
@show resβ(β)

# compare mathematical and numerical gradient
[vec(∇resβ(β)) vec(∇resβ_autodiff(β))]

resβ(β) = [0.8012638765796852, -6.734952547066679, -1.3994071698413866, -0.48023808695797]


12×2 Matrix{Float64}:
 -1.07727    -1.07727
 -3.65238    -3.65238
 -1.22049    -1.22049
 -1.02842    -1.02842
  2.23488     2.23488
 -7.11068    -7.11068
  0.0987079   0.0987079
  0.159001    0.159001
 -1.00397    -1.00397
 -8.25796    -8.25796
 -1.46237    -1.46237
  0.0784727   0.0784727

## Is Gradient of likelihood correct?

In [7]:
# sample data
gc = qc_model.data[1]
X = qc_model.data[1].X # d by p
y = qc_model.data[1].y # d by 1
θ = qc_model.θ

function A_mul_b!(c::AbstractVector{T}, A::AbstractMatrix, b::AbstractVector) where T
    n, p = size(A)
    fill!(c, zero(T))
    for j in 1:p, i in 1:n
        c[i] += A[i, j] * b[j]
    end
    return c
end

# assumes y, X are available
function autodiff_loglikelihood_bernoulli(β::AbstractVector{T}) where T
    n, p = size(X)
    # allocate vector of type T
    η = zeros(T, n)
    μ = zeros(T, n)
    varμ = zeros(T, n)
    dμ = zeros(T, n)
    w1 = zeros(T, n)
    w2 = zeros(T, n)
    res = zeros(T, n)
    ∇resβ = zeros(T, n, p)
    storage_n = zeros(T, n)
    q = zeros(T, gc.m)
    # update_res! step (need to avoid BLAS)
    A_mul_b!(η, X, β)
    for i in 1:gc.n
        μ[i] = GLM.linkinv(gc.link, η[i])
        varμ[i] = GLM.glmvar(gc.d, μ[i]) # Note: for negative binomial, d.r is used
        dμ[i] = GLM.mueta(gc.link, η[i])
        w1[i] = dμ[i] / varμ[i]
        w2[i] = w1[i] * dμ[i]
        res[i] = y[i] - μ[i]
    end
    # standardize_res! step
    for j in eachindex(y)
        res[j] /= sqrt(varμ[j])
    end
    # std_res_differential! step (this will compute ∇resβ)
    for i in 1:gc.p
        for j in 1:gc.n
            ∇resβ[j, i] = -sqrt(varμ[j]) * X[j, i] - (0.5 * res[j] * (1 - (2 * μ[j])) * X[j, i])
        end
    end
    # update Γ
    @inbounds for k in 1:gc.m
        A_mul_b!(storage_n, gc.V[k], res)
        q[k] = dot(res, storage_n) / 2 # q[k] = 0.5 r' * V[k] * r (update variable b for variance component model)
    end
    # component_loglikelihood
    logl = zero(T)
    for j in 1:gc.n
        logl += QuasiCopula.loglik_obs(gc.d, y[j], μ[j], one(T), one(T))
    end
    tsum = dot(θ, gc.t)
    logl += -log(1 + tsum)
    qsum  = dot(θ, q) # qsum = 0.5 r'Γr
    logl += log(1 + qsum)
    return logl
end

# autodiff Gradient
∇logl_bernoulli = x -> ForwardDiff.gradient(autodiff_loglikelihood_bernoulli, x)
∇logl_bernoulli(qc_model.β)


3-element Vector{Float64}:
 -1.0570025767760889
 -2.218682510700683
  0.18209959353709226

In [8]:
loglikelihood!(qc_model, true, false)
qc_model.data[1].∇β

3-element Vector{Float64}:
 -1.0570025767760889
 -2.2186825107006833
  0.1820995935370926

# Is Hessian of loglikelihood correct?

In [9]:
# autodiff Hessian
ForwardDiff.jacobian(
    x -> ForwardDiff.jacobian(autodiff_loglikelihood_bernoulli, x), qc_model.β
)

LoadError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{typeof(loglikelihood), ForwardDiff.Dual{ForwardDiff.Tag{var"#11#12", Float64}, Float64, 3}}, ForwardDiff.Dual{ForwardDiff.Tag{var"#11#12", Float64}, Float64, 3}, 3})
[0mClosest candidates are:
[0m  (::Type{T})(::Real, [91m::RoundingMode[39m) where T<:AbstractFloat at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/rounding.jl:200
[0m  (::Type{T})(::T) where T<:Number at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/boot.jl:770
[0m  (::Type{T})([91m::VectorizationBase.Double{T}[39m) where T<:Union{Float16, Float32, Float64, VectorizationBase.Vec{<:Any, <:Union{Float16, Float32, Float64}}, VectorizationBase.VecUnroll{var"#s36", var"#s35", var"#s34", V} where {var"#s36", var"#s35", var"#s34"<:Union{Float16, Float32, Float64}, V<:Union{Bool, Float16, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, SIMDTypes.Bit, Vectorizatio

In [8]:
# autodiff Hessian
autodiff_hessian = x -> ForwardDiff.hessian(loglikelihood, x)
H_autodiff = autodiff_hessian(qc_model.β)

LoadError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{typeof(loglikelihood), Float64}, ForwardDiff.Dual{ForwardDiff.Tag{typeof(loglikelihood), Float64}, Float64, 3}, 3})
[0mClosest candidates are:
[0m  (::Type{T})(::Real, [91m::RoundingMode[39m) where T<:AbstractFloat at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/rounding.jl:200
[0m  (::Type{T})(::T) where T<:Number at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/boot.jl:770
[0m  (::Type{T})([91m::VectorizationBase.Double{T}[39m) where T<:Union{Float16, Float32, Float64, VectorizationBase.Vec{<:Any, <:Union{Float16, Float32, Float64}}, VectorizationBase.VecUnroll{var"#s36", var"#s35", var"#s34", V} where {var"#s36", var"#s35", var"#s34"<:Union{Float16, Float32, Float64}, V<:Union{Bool, Float16, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, SIMDTypes.Bit, VectorizationBase.AbstractSIMD{var"#s35", var"#s34"}}}} at ~/.

### Hessian evaluated with 2 terms only

In [106]:
n, q = size(G)
p = length(qc_model.β)
T = eltype(qc_model.data[1].X)
P = zeros(T, p, p)
for i in 1:n
    gc = qc_model.data[i]
    d = gc.n # number of observations for current sample
    # GLM term
    P += Transpose(gc.X) * Diagonal(gc.w2) * gc.X
    # trailing terms
    res = gc.res # d × 1 standardized residuals
    ∇resβ = gc.∇resβ # d × p
    Γ = zeros(T, d, d)
    for k in 1:gc.m # loop over variance components
        Γ .+= qc_model.θ[k] .* gc.V[k]
    end
    denom = abs2(1 + 0.5 * (res' * Γ * res))
    P += (∇resβ' * Γ * res) * (∇resβ' * Γ * res)' / denom
end
P

3×3 Matrix{Float64}:
  7896.6    2695.55  -3169.72
  2695.55   8402.23  -3442.68
 -3169.72  -3442.68   9146.81